# Resolução do Professor - Análise Cruzada Otimizada

---

## Bloco 1 - Importação Simplificada

**Abordagem Direta**

In [1]:
import pandas as pd

aposentadorias = pd.read_csv('data/aposentadorias.csv', sep=';', encoding='latin-1', skiprows=2)
devedores = pd.read_csv('data/devedores.csv', sep=';', encoding='latin-1')

Diferenças da abordagem:

- Nomenclatura mais limpa: Sem sufixo `_df_original`

- Importações mínimas: Apenas pandas (sem numpy desnecessário)

- Foco na simplicidade: Código mais direto

---

# Bloco 2 - Análise Exploratória Inicial

**Compreensão dos Dados**

In [2]:
aposentadorias.describe()

,UORG,Referï¿½ncia,Nï¿½vel,Ingresso no Serviï¿½o Pï¿½blico Civil,Data de Publicaï¿½ï¿½o
count,5405.0,0.0,0.0,0.0,43.000000
mean,25201.0,NaN,NaN,NaN,38211.302326
std,0.0,NaN,NaN,NaN,945.571043
min,25201.0,NaN,NaN,NaN,37305.000000
25%,25201.0,NaN,NaN,NaN,37740.000000
50%,25201.0,NaN,NaN,NaN,37740.000000
75%,25201.0,NaN,NaN,NaN,38880.500000
max,25201.0,NaN,NaN,NaN,40848.000000


In [3]:
devedores.describe()

,DATA DE GERAÇÃO,CPF/CNPJ DEVEDOR,TIPO DE PESSOA,TIPO DO DEVEDOR,NOME DO DEVEDOR,NÚMERO DO TERMO DE INSCRIÇÃO,DATA DA INSCRIÇÃO,SEQUENCIAL DO CRÉDITO,SITUAÇÃO DO CRÉDITO,MOTIVO DA SUSPENSÃO,ORIGEM,CREDOR,SALDO DEVEDOR SEM HONORÁRIOS,OBSERVAÇÃO
count,2576,2568,2568,2576,2576,2576,2576,2576,2576,225,2576,2576,2576,13
unique,1,2238,2,2,2241,2306,1398,2315,4,4,4,4,2072,1
top,04/12/2023,XXX.827.945-XX,Pessoa jurídica,Principal,Stephane Joseph Ghislain Mery de Montigny,0002/1984,10/06/2003,CR2013/0005435,Em execução,É suspensa a cobrança por força de depósito in...,Multa,Banco Central,"R$ 38.692,27",O saldo devedor sem honorários não correspond...
freq,2576,17,1669,2315,17,6,23,6,2090,74,2358,2555,16,13


In [5]:
# Verificação de tipos de dados
type(devedores['CPF/CNPJ DEVEDOR'][0])

str

Estratégia de exploração:

- `.describe()`: Visão estatística geral

- Verificação de tipos: Entender formato dos dados

- Análise antes da transformação: Conhecer estrutura original

---

## Bloco 3 - Filtragem Eficiente

**Seleção de Pessoas Físicas**

In [7]:
devedores_cpf = devedores[devedores['TIPO DE PESSOA'] == "Pessoa física"]
# Alternativa comentada: devedores.query('`TIPO DE PESSOA` == "Pessoa física"')

devedores_cpf.count()

DATA DE GERAÇÃO                 899
CPF/CNPJ DEVEDOR                899
TIPO DE PESSOA                  899
TIPO DO DEVEDOR                 899
NOME DO DEVEDOR                 899
NÚMERO DO TERMO DE INSCRIÇÃO    899
DATA DA INSCRIÇÃO               899
SEQUENCIAL DO CRÉDITO           899
SITUAÇÃO DO CRÉDITO             899
MOTIVO DA SUSPENSÃO             100
ORIGEM                          899
CREDOR                          899
SALDO DEVEDOR SEM HONORÁRIOS    899
OBSERVAÇÃO                        1
dtype: int64

Vantagens da abordagem:

- Filtragem prévia: Reduz volume de dados para processamento

- Foco no target: Apenas pessoas físicas (CPF, não CNPJ)

- Eficiência: Menos dados = processamento mais rápido

💡 Insight: Filtrar primeiro, processar depois!

---

## Bloco 4 - Extração de CPF com String Slicing

**Operação Vetorizada Nativa**

In [10]:
devedores_cpf['CPF/CNPJ DEVEDOR'] = devedores_cpf['CPF/CNPJ DEVEDOR'].str[3:12]


C:\Users\ana.leandro\AppData\Local\Temp\ipykernel_33200\2430731822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  devedores_cpf['CPF/CNPJ DEVEDOR'] = devedores_cpf['CPF/CNPJ DEVEDOR'].str[3:12]


In [11]:
devedores_cpf['CPF/CNPJ DEVEDOR']

1       .160.007-
6       .340.947-
9       .028.317-
10      .993.977-
11      .402.247-
          ...    
2563    .619.996-
2564    .453.768-
2566    .875.635-
2568    .728.571-
2571    .858.898-
Name: CPF/CNPJ DEVEDOR, Length: 899, dtype: object

In [8]:
# Versão corrigida:
devedores_cpf_copy = devedores[devedores['TIPO DE PESSOA'] == "Pessoa física"].copy()

devedores_cpf_copy['CPF/CNPJ DEVEDOR'] = devedores_cpf['CPF/CNPJ DEVEDOR'].str[3:12]

Análise da abordagem:

- `.str[3:12]`: Extrai caracteres das posições 3 a 11 (9 dígitos)

- Vetorização nativa: Pandas otimizado, não `.apply()`

- Formato CPF: Remove prefixos/formatação, mantém apenas números

---

## Bloco 5 - Padronização Paralela

**Mesmo Tratamento para Ambas as Bases**

In [12]:
aposentadorias['CPF'] = aposentadorias['CPF'].str[3:12]

In [13]:
aposentadorias['CPF']

0       .161.491-
1       .297.631-
2       .895.388-
3       .983.931-
4       .347.385-
          ...    
5400    .533.991-
5401    .454.201-
5402    .828.526-
5403    .113.787-
5404    .805.121-
Name: CPF, Length: 5405, dtype: object

Consistência:

- Mesmo padrão: Ambas as bases com CPF extraído

- Formato uniforme: 9 dígitos centrais do CPF

- Preparação para merge: Chaves compatíveis

---

## Bloco 6 - Tentativa de Merge Inicial

**Teste com Múltiplas Chaves**

In [49]:
pd.merge(
    aposentadorias, 
    devedores, 
    left_on=['CPF', 'Nome'], 
    right_on=['CPF/CNPJ DEVEDOR', 'NOME DO DEVEDOR']
)

,Nome,CPF,Situaï¿½ï¿½o Vï¿½nculo,Matrï¿½cula,UORG,ï¿½rgï¿½o,ï¿½rgï¿½o Superior,Classe,Padrï¿½o,Referï¿½ncia,...,NOME DO DEVEDOR,NÚMERO DO TERMO DE INSCRIÇÃO,DATA DA INSCRIÇÃO,SEQUENCIAL DO CRÉDITO,SITUAÇÃO DO CRÉDITO,MOTIVO DA SUSPENSÃO,ORIGEM,CREDOR,SALDO DEVEDOR SEM HONORÁRIOS,OBSERVAÇÃO


Estratégia:

- Merge por CPF + Nome: Dupla validação

- Nomes de colunas diferentes: `left_on` vs `right_on`

- Teste exploratório: Verificar se há correspondências

⚠️ Problema: Nomes das colunas e formatos diferentes impedem matches.

---

## Bloco 7 - Padronização de Colunas

**Renomeação para Compatibilidade**

In [ ]:
devedores_cpf.rename(
    columns={
        'CPF/CNPJ DEVEDOR': 'CPF',
        'NOME DO DEVEDOR': 'Nome'
    },
    inplace=True
)

C:\Users\ana.leandro\AppData\Local\Temp\ipykernel_28580\656085705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  devedores_cpf.rename(


Vantagens do `.rename()`:

- Padronização: Nomes de colunas iguais

- `inplace=True`: Modifica DataFrame original (economia de memória)

- Preparação para merge: Chaves com nomes idênticos

---

## Bloco 8 - Padronização de Nomes

**Normalização de Case**

In [52]:
aposentadorias['Nome'] = aposentadorias['Nome'].str.upper()
aposentadorias['Nome']

0                    ABADIA MARCIA CUNHA
1           ABDIAS CASSIO MARROCOS GOMES
2             ABDIEL ANDRIOLO DE ANDRADE
3                 ABDON LOPES DE MENESES
4                     ABEL ALFREDO FILHO
                      ...               
5400               ZULEICA DE MELO TELES
5401           ZULEICA MARIA SOUZA PORTO
5402                ZULMA GONCALVES DIAS
5403                       ZULMAR MAYETA
5404    ZULMIRA MARIA BERNARDES NORMANDO
Name: Nome, Length: 5405, dtype: object

In [53]:
devedores_cpf['Nome'] = devedores_cpf['Nome'].str.upper()
devedores_cpf['Nome']

C:\Users\ana.leandro\AppData\Local\Temp\ipykernel_28580\877751381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  devedores_cpf['Nome'] = devedores_cpf['Nome'].str.upper()


1           JOSE CARLOS DA SILVA GOMES
6              ANDRE DE SOUZA OLIVEIRA
9                ADRIANO DIAS OLIVIERI
10               EDUARDO DIAS OLIVIERI
11                     INGRID TEIXEIRA
                     ...              
2563                  SARA ALVES GOMES
2564       GERSON LUIZ MENDES DE BRITO
2566    WALTER FERNANDEZ ALVAREZ FILHO
2568        DORIVAN DELMOND DE GOUVEIA
2571         DEBORAH REGINA FRANCISCON
Name: Nome, Length: 899, dtype: object

Operação vetorizada:

- `.str.upper()`: Converte para maiúsculas

- Consistência: Evita problemas de case sensitivity

- Nativo do Pandas: Otimizado, sem loops Python

---

## Bloco 9 - Merge Final Otimizado

**União Eficiente**

In [54]:
pd.merge(
    aposentadorias, 
    devedores_cpf, 
    left_on=['CPF', 'Nome'], 
    right_on=['CPF', 'Nome'],
    how='inner'
)

,Nome,CPF,Situaï¿½ï¿½o Vï¿½nculo,Matrï¿½cula,UORG,ï¿½rgï¿½o,ï¿½rgï¿½o Superior,Classe,Padrï¿½o,Referï¿½ncia,...,TIPO DO DEVEDOR,NÚMERO DO TERMO DE INSCRIÇÃO,DATA DA INSCRIÇÃO,SEQUENCIAL DO CRÉDITO,SITUAÇÃO DO CRÉDITO,MOTIVO DA SUSPENSÃO,ORIGEM,CREDOR,SALDO DEVEDOR SEM HONORÁRIOS,OBSERVAÇÃO
0,DAISY MARIA GUSMAO WELLISCH,.976.541-,Aposentado,222*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAC,III,NaN,...,Principal,2020.001-060,27/05/2020,CR2020/0000180,Suspenso,Outros motivos,Relativos a servidor,Banco Central,"R$ 84.704,26",NaN
1,DALTON LEAL DIMA,.952.137-,Aposentado,223*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,IV,NaN,...,Principal,2020.001-079,16/09/2020,CR2020/0000306,Suspenso,Decisão judicial,Relativos a servidor,Banco Central,"R$ 74.057,35",NaN
2,DEJAIR CARLOS CARVALHO,.301.821-,Aposentado,235*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,II,NaN,...,Principal,2017.001-137,10/08/2017,CR2017/0000669,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 35.692,30",NaN
3,ELMO CAVALCANTE GOMES,.823.827-,Aposentado,285*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,IV,NaN,...,Principal,2017.001-139,10/08/2017,CR2017/0000671,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 63.190,56",NaN
4,EVERALDO JOSE DA SILVA JUNIOR,.013.866-,Aposentado,312*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,IV,NaN,...,Principal,2019.001-155,30/07/2019,CR2019/0000398,Suspenso,Outros motivos,Relativos a servidor,Banco Central,"R$ 40.881,42",NaN
5,FRANCISCO CARLOS SERRANO,.163.608-,Aposentado,337*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCJURPE,I,NaN,...,Principal,2020.001-076,28/08/2020,CR2020/0000277,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 110.139,59",NaN
6,FRANCISCO MACHADO CARNEIRO,.461.231-,Aposentado,344*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPTE,IV,NaN,...,Principal,2016.001-093,09/06/2016,CR2016/0000440,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 93.983,04",NaN
7,IVAN RAMOS CASTRO,.982.677-,Aposentado,449*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,IV,NaN,...,Principal,2017.001-150,10/08/2017,CR2017/0000682,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 35.383,53",NaN
8,JANETE RIBEIRO DOS REIS,.272.114-,Aposentado,462*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAA,III,NaN,...,Principal,2017.001-216,11/08/2017,CR2017/0000815,Em execução,NaN,Relativos a servidor,Banco Central,"R$ 7.299,35",NaN
9,JOAO CARLOS TEIXEIRA DOS SANTOS,.454.727-,Aposentado,481*****,25201,BANCO CENTRAL DO BRASIL,BANCO CENTRAL DO BRASIL,BCESPAE,IV,NaN,...,Principal,2017.001-209,11/08/2017,CR2017/0000807,Suspenso,Decisão judicial,Relativos a servidor,Banco Central,"R$ 29.336,99",NaN


Parâmetros finais:

- Chaves idênticas: `left_on` = `right_on`

- `how='inner'`: Apenas registros presentes em ambas as bases

- Dupla validação: CPF + Nome para maior precisão

---

## Comparação: Minha Solução vs Professor

### 1. Tabela Comparativa

| Aspecto  | Minha Abordagem | Abordagem do Professor |
| -------- |:---------------:|:----------------------:|
| Padronização CPF | Função .apply() | .str[3:12] vetorizado |
| Validação Nomes | Função complexa de similaridade | .str.upper() simples |
| Intersecção      | set() + &     | pd.merge() direto |
| Filtragem     |  Múltiplos passos    | Filtragem prévia eficiente |
| Linhas de código      | ~40 linhas     | ~15 linhas
| Performance      | Loops Python (lento)     | Vetorização Pandas (rápido) |
| Complexidade      | Alta    | Baixa |
| Manutenibilidade      | Difícil     | Fácil |



### 2. Resumo das Lições Aprendidas

**Principais Insights da Trilha de Dados**

**Fundamentos sólidos são essenciais**: NumPy é a base que torna Pandas possível - compreender vetorização e arrays multidimensionais é fundamental para análise de dados eficiente.


**Exploração estruturada**: `.head()`, `.describe()`, `.info()` e `.unique()` devem ser seus primeiros comandos em qualquer dataset novo.


**Indexação importa**: Dominar `.loc[]` vs `.iloc[]` e entender o `SettingWithCopyWarning` evita bugs silenciosos e código não-maintível.


**Vetorização é rei**: Substitua `.apply()` por operações nativas (`.str.upper()`, `.str[3:12]`, operadores matemáticos). A diferença pode ser de até **25x+ em performance**.


**Filtre primeiro, processe depois**: Reduzir o volume de dados antes das transformações economiza tempo e recursos computacionais.


**Simplicidade vence complexidade**: A solução do professor demonstrou que menos código frequentemente significa melhor performance. Sempre questione: "O Pandas já faz isso nativamente?"